This notebook is using the ARIMA (AutoRegressive Integrated Moving Average) model from the statsmodels library to forecast time series data, specifically minimum liquid assets.  This traditional model serves as a benchmark model to the Temporal Fusion Transformer model.

In [ ]:
from __future__ import annotations # allows multiple typing of arguments in Python versions prior to 3.10

In [ ]:
# Code below included to ensure compatibility with scikit-learn v1.1.x
from sklearn import set_config
set_config(display='text')

In [ ]:
#| include: false
from nbdev.showdoc import show_doc

In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types as ptypes
from gingado.model_documentation import ggdModelDocumentation, ModelCard
from gingado.utils import read_attr
from sklearn.base import BaseEstimator
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.utils.metaestimators import available_if
from sklearn.utils.validation import check_is_fitted
import warnings

In [ ]:
df_new = pd.read_csv("../Data/final.csv")

In [ ]:
unnamedCols = [col for col in df_new.columns if 'Unnamed' in col]
df_new.drop(columns=unnamedCols, inplace=True)

In [ ]:
df_new['CORE_DEPOSITS']=df_new.apply(lambda x: x['01_CURR_ACC']+x['03_SAVINGS']+x['04_OTHER_DEPOSITS'], axis=1)
df_new['TOTAL_DEPOSITS']=df_new.apply(lambda x: x['05_BANKS_DEPOSITS']+x['XX_CUSTOMER_DEPOSITS'], axis=1)
df_new['GL_TO_TOTAL_FUNDING']=df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['F125_LIAB_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_FUNDING']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F125_LIAB_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_ASSET']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_DEPOSIT']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['TOTAL_DEPOSITS']+0.0001), axis=1)
df_new['LiqAsset2DemandLiab']=df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['ExcessShortTLiab2LongTAsset']=df_new.apply(lambda x: (x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']-x['XX_TOTAL_LIQUID_ASSET'])/(x['F077_ASSETS_TOTAL'] - x['XX_TOTAL_LIQUID_ASSET']+0.0001), axis=1)
df_new['CD_TO_TOTAL_ASSET']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['GL_TO_TOTAL_DEPOSITS']=df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['LIQASSET2TOTALASSET']=df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['F077_ASSETS_TOTAL'] +0.0001), axis=1)
df_new['BANKSIZE']= np.log10(df_new['F077_ASSETS_TOTAL'])
df_new['LOAN2DEPOSIT']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT'] +0.0001), axis=1)
df_new['LIQASSET2DEPOSIT']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['CURRENTRATIO']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['XX_TOTAL_LIQUID_LIAB'] + 0.0001), axis=1)
df_new['LIQASSET2TOTALASSET']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['F077_ASSETS_TOTAL'] + 0.0001), axis=1)
df_new['VOLATILEDEPOSITS2LIAB']= df_new.apply(lambda x: (x['XX_TOTAL_LIQUID_ASSET']-x['CORE_DEPOSITS'])/(x['F125_LIAB_TOTAL'] + 0.0001), axis=1)
df_new['LOAN2ASSETS']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['DOMESTICDEPOSIT2ASSETS']= df_new.apply(lambda x: (x['TOTAL_DEPOSITS']-x['10_FOREIGN_DEPOSITS_AND_BORROWINGS'])/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['LOAN2COREDEPOSIT']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS']+0.0001), axis=1)
df_new['BOTBAL2TOTALDEPOSIT']= df_new.apply(lambda x: x['XX_BOT_BALANCE']/(x['TOTAL_DEPOSITS']+0.0001), axis=1)
df_new['REPORTINGDATE'] = pd.to_datetime(df_new.REPORTINGDATE)

/var/folders/cd/427m1yk53zsfjp7fxx6bb17h0000gn/T/ipykernel_42572/268649099.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_new['REPORTINGDATE'] = pd.to_datetime(df_new.REPORTINGDATE)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima.model import ARIMAResults
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
df_new = df_new[['REPORTINGDATE','INSTITUTIONCODE','XX_MLA']].set_index('REPORTINGDATE')
df_new

,INSTITUTIONCODE,XX_MLA
REPORTINGDATE,,
2010-05-02,B5014,86.898135
2010-12-02,B5014,89.545334
2010-02-19,B5014,90.997005
2010-02-26,B5014,83.580184
2010-05-03,B5014,80.905077
...,...,...
2021-10-12,B5919,52.676231
2021-12-17,B5919,61.124212
2021-12-24,B5919,61.384902


In [ ]:
def RMSE(data,cutoff,p,d,q):
 with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  try:
   X_train = data[data.index <cutoff]
   X_test = data[data.index >=cutoff]
   model = ARIMA(X_train, order=(p,d,q))
   results = model.fit()
   fitted = results.apply(X_test)
   df_ = fitted.fittedvalues
   df_.iloc[0] = results.forecast(1)
   _df = pd.concat([X_test,df_],axis=1)
   _df.columns = ["actual", "fitted"]
   _df['difference'] = (_df.actual-_df.fitted)**2
   return np.sqrt(_df.difference.mean())
  except:
   np.nan



# plot forecasts against actual outcomes

In [ ]:
df_new['RMSE'] =df_new.groupby('INSTITUTIONCODE').transform(lambda x: RMSE(x,'2019-01-01',1,1,1))

/var/folders/cd/427m1yk53zsfjp7fxx6bb17h0000gn/T/ipykernel_42572/3931336517.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new['RMSE'] =df_new.groupby('INSTITUTIONCODE').transform(lambda x: RMSE(x,'2019-01-01',1,1,1))


In [ ]:
df_new.RMSE.dropna().unique().mean()

13.27221080054615

In [ ]:
data = df_new[df_new.INSTITUTIONCODE=='B5014'][['XX_MLA']]
X_train = data[data.index <'2019-01-01']
X_test = data[data.index >='2019-01-01']


# Model Overview
This section creates and fits an ARIMA model to the training data (`X_train`) with a specified order of (1,1,1).
It forecasts future values using the fitted model, then calculates and stores the squared difference between
the actual values (`X_test`) and the fitted values. Finally, it computes the root mean squared error (RMSE)
to evaluate the model's prediction accuracy.

In [ ]:
model = ARIMA(X_train, order=(1,1,1))
results = model.fit()
fitted = results.apply(X_test)
df_ = fitted.fittedvalues
df_.iloc[0] = results.forecast(1)
_df = pd.concat([X_test,df_],axis=1)
_df.columns = ["actual", "fitted"]
_df['difference'] = (_df.actual-_df.fitted)**2
np.sqrt(_df.difference.mean())

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.9/site-packages/statsmodels

11.514077124419153

InvalidIndexError: (slice(None, None, None), slice(1, None, None))